# 4.2 딥러닝 구조



## 4.2.1 딥러닝 용어

In [1]:
import torch

### 활성화 함수
전달받은 값을 출력할 때 일정 기준에 따라 출력 값을 변화시키는 비선형 함수

In [ ]:
# 활성화 함수
# ReLU 함수(입력<0 -> 0, 입력>0 -> x)와 softmax 함수(입력이 0~1 사이에 출력되도록 정규화. 출력값 총합이 항상 1) 구현 예시
class Net(torch.nn.Module):
    def __init__(self, n_feature,n_hidden,n_output):
        super(Net,self).__init__()
        self.hidden=torch.nn.Linear(n_feature, n_hidden) #은닉층
        self.relu=torch.nn.ReLu(inplace=True)
        self.out=torch.nn.Linear(n_hidden,n_output) #출력층
        self.softmax=torch.nn.Softmax(dim=n_output)

    def forward(self,x):
        x=self.hidden(x)
        x=self.relu(x) #은닉층을 위한 렐루 활성화 함수
        x=self.out(x)
        x=self.softmax(x) #출력층을 위한 소프트맥스 활성화 함수
        return x

### 손실 함수
학습을 통해 얻은 데이터의 추정치가 실제 데이터와 얼마나 차이가 나는지 평가하는 지표

In [ ]:
# 손실함수로 MSE(Mean Squared Error) 적용
loss_fn=torch.nn.MSELoss(reduction='sum')
y_pred=model(x)
loss=loss_fn(y_pred,y)

In [1]:
# 손실함수로 CEE(Cross Entropy Loss) 적용
# CEE는 분류 문제어서 one-hot encoding을 했을 때만 사용 가능
import torch.nn as nn
loss=nn.CrossEntropyLoss()
input=torch.randn(5,6,requires_grad=True) #torch.randn은 평균이 0이고 표준편차가 1인 가우시안 정규분포를 이용하여 숫자 생성
target=torch.empty(3,dtype=torch.long).random_(5) #torch.empty는 dtype torch.float32의 랜덤한 값으로 채워진 텐서를 반환
output=loss(input,target)
output.backward()

## 4.2.3 딥러닝의 문제점과 해결 방안

### 드롭아웃
과적합 방지 목적으로 학습 과정 중 임의로 일부 노드들을 학습에서 제외

In [ ]:
# 드롭아웃 구현 예시
class DropoutModel(torch.nn.Module):
    def __init__(self):
        super(DropoutModel, self).__init__()
        self.layer1=torch.nn.Linear(784,1200)
        #50의 노드를 무작위로 선택하여 사용하지 않겠다는 의미
        self.dropout1=torch.nn.Dropout(0.5)
        self.layer2=torch.nn.Linear(1200,1200)
        self.dropout2=torch.nn.Dropout(0.5)
        self.layer3=torch.nn.Linear(1200,10)

    def forward(self,x):
        x=F.relu(self.layer1(x))
        x=self.dropout1(x)
        x=F.relu(self.layer2(x))
        x=self.dropout2
        return self.layer3(x)

### Mini-batch gradient descent
전체 데이터셋을 미니 배치 여러 개로 나눠 미니 배치 1개마다 기울기를 구한 후 그것의 평균 기울기를 이용해 모델을 업데이트해서 학습

In [ ]:
# mini-batch gradient descent 구현 예시
class CustomDataset(Dataset):
    def __init__(self):
        self.x_data=[[1,2,3],[4,5,6],[7,8,9]]
        self.y_data=[[12],[18],[11]]
        def __len__(self):
            return len(self.x_data)
        def __getitem__(self,idx):
            x=torch.FloatTensor(self.x_data[idx])
            y=torch.FloatTensor(self.y_data[idx])
            return x,y

dataset=CustomDataset()
dataloader=DataLoader(
    dataset, #데이터셋
    batch_size=2, #미니 배치 크기로 2의 제곱수를 사용
    shuffle=True, #데이터를 불러올 때마다 랜덤으로 섞어서 가져오기
)

### Optimizer
확률적 경사 하강법의 파라미터 변경 폭이 불안정한 문제를 해결하기 위해 학습 속도와 운동량을 조정

In [ ]:
# Adagrad
optimizer=torch.optim.Adagrad(model.parameters(),lr=0.01) #lr: learning rate default=1e-2

In [ ]:
# Adadelta
optimizer=torch.optim.Adadelta(model.parameters(),lr=1.0) #lr: learning rate default=1.0

In [ ]:
# RMSProp
optimizer=torch.optim.RMSprop(model.parameters(),lr=0.01) #lr: learning rate default=1e-2

In [ ]:
# SGD Momentum(SGD with Momentum)
optimizer=torch.optim.SGD(model.parameters(),lr=0.01, momentum=0.9)

In [ ]:
# NAG(Nesterov Accerlerated Gradient)
optimizer=torch.optim.SGD(model.parameters(),lr=0.01, momentum=0.9, nesterov=True) #nesterov default: False

In [ ]:
# Adam
optimizer=torch.optim.Adam(model.parameters(),lr=0.01) #lr: learning rate default=1e-3